<a href="https://colab.research.google.com/github/soumilhooda/MLDLNNtoCV/blob/main/Q7_NNFL_Assignment2_SoumilHooda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving data5.xlsx to data5.xlsx


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix as cm
from numpy import random

In [3]:
Dataset = pd.read_excel('data5.xlsx', header = None)
Dataset = np.array(Dataset)
np.random.shuffle(Dataset)

def initialize(Dataset):
  X = np.array(Dataset[:,:-1], dtype = float)
  y = np.array(Dataset[:,-1], dtype = int)
  X = (X - X.mean(axis = 0))/X.std(axis = 0)
  return X, y

X_total, y_total = initialize(Dataset)
y = np.zeros((len(y_total), 3))
for i in range(len(y_total)):
  if y_total[i] == 1:
    y[i,0] = 1.0
  elif y_total[i] == 2:
    y[i,1] = 1.0
  elif y_total[i] == 3:
    y[i,2] = 1.0

X_Train, Y_Train = X_total[:int(0.7 * len(X_total))], y[:int(0.7 * len(X_total))]
X_validation, y_validation = X_total[int(0.7 * len(X_total)):], y[int(0.7 * len(X_total)):]
alpha = 0.5

In [4]:
def sigmoid(x, derivative = False):
  if derivative == True:
    return x * (1 - x)
  return 1/(1 + np.exp(-x))

In [5]:
def tanh(x):
  return np.tanh(x)

In [6]:
def cost_calc(neuralNetwork, x, y):
  cost = 0
  for i in range(len(x)):
    X = np.reshape(x[i], (len(x[i]), 1))
    cost += 0.5/len(x) * np.sum((y[i] - neuralNetwork.forward_propagation(X)) ** 2)
  return cost

In [7]:
class NeuralNetwork(object):
  def __init__(self, sizes):
    self.number_layers = len(sizes)
    self.sizes = sizes
    self.weight = {}
    self.activation = {} 
    self.bias = {} 

    for i in range(1, self.number_layers):
      self.weight[i] = np.random.randn(self.sizes[i-1], self.sizes[i])
    
    for i in range(1, self.number_layers):
      self.bias[i] = np.random.randn(self.sizes[i], 1)

    for i in range(1, self.number_layers):
      self.activation[i] = np.zeros([self.sizes[i], 1])
  
  def forward_propagation(self, X):
    self.activation[0] = X
    for i in range(1, self.number_layers):
      self.activation[i] = sigmoid(np.dot(self.weight[i].T, self.activation[i-1]) + self.bias[i])
    return self.activation[self.number_layers - 1]
  
  def backward_propagation(self, X, y, output):
    self.d = {}
    self.d_output = (y - output) * sigmoid(output, derivative = True)
    self.d[self.number_layers - 1] = self.d_output

    for i in range(self.number_layers - 1, 1, -1):
      self.d[i-1] = np.dot(self.weight[i], self.d[i]) * sigmoid(self.activation[i-1], derivative = True)

    for i in range(1, self.number_layers - 1):
      self.weight[i] += alpha * np.dot(self.activation[i-1], self.d[i].T)
    
    for i in range(1, self.number_layers - 1):
      self.bias[i] += alpha * self.d[i]
        
  def training_process(self, X, y):
    X = np.reshape(X, (len(X), 1))
    output = self.forward_propagation(X)
    self.backward_propagation(X, y, output)

  def get_weight(self):
    return self.weight
  
  def load_weights(self, weight):
    self.weight = weight

  def get_activation(self, X):
    X = np.reshape(X, (len(X), 1))
    self.forward_propagation(X)
    return self.activation
  
  def load_activations(self, activation):
    self.activation = activation

In [8]:
autoencoder1 = NeuralNetwork([7,6,7])
autoencoder2 = NeuralNetwork([6,3,6])

for i in range(500):
  for j, row in enumerate(X_Train):
    row = np.reshape(row, (7,1))
    autoencoder1.training_process(row, row)
  
  cost = cost_calc(autoencoder1, X_Train, X_Train)

   

autoencoder2_input = []
for row in X_Train:
  autoencoder2_input.append(autoencoder1.get_activation(row)[1])

autoencoder2_input = np.array(autoencoder2_input)


for i in range(500):
  for j, row in enumerate(autoencoder2_input):
    row = np.reshape(row, (6, 1))
    autoencoder2.training_process(row, row)
  
  cost = cost_calc(autoencoder2, autoencoder2_input, autoencoder2_input)

elm_input = []
for row in autoencoder2_input:
  elm_input.append(autoencoder2.get_activation(row)[1])
elm_input = np.array(elm_input)

elm_neurons = 25
output_neurons = 3
weight_elm = np.random.randn(elm_input.shape[1], elm_neurons)

np.random.seed(1)
elm_input = np.reshape(elm_input, (147, 3))
H = np.matmul(elm_input, weight_elm)
H = tanh(H)
H_inv = np.linalg.pinv(H)
weight_final = np.matmul(H_inv, Y_Train)

layer1_output = []
for i, row in enumerate(X_validation):
  activation = autoencoder1.get_activation(row)[1]
  layer1_output.append(activation)
layer1_output = np.array(layer1_output)
layer1_output = np.reshape(layer1_output, (63, 6))

layer2_output = []
for i, row in enumerate(layer1_output):
  activation = autoencoder2.get_activation(row)[1]
  layer2_output.append(activation)
layer2_output = np.array(layer2_output)
layer2_output = np.reshape(layer2_output, (63, 3))

ht = np.matmul(layer2_output, weight_elm)
ht = tanh(ht)
y_prediction = np.matmul(ht, weight_final)

activation = [np.argmax(y_prediction[i]) for i in range(len(y_prediction))]
bias = [np.argmax(y_validation[i]) for i in range(len(y_validation))]

confusion_matrix = cm(bias, activation)

confusion_matrix = np.array(confusion_matrix)

accuracy1 = (confusion_matrix[0][0])/(confusion_matrix[0][0] + confusion_matrix[0][1] + confusion_matrix[0][2])
accuracy2 = (confusion_matrix[1][1])/(confusion_matrix[1][0] + confusion_matrix[1][1] + confusion_matrix[1][2])
accuracy3 = (confusion_matrix[2][2])/(confusion_matrix[2][0] + confusion_matrix[2][1] + confusion_matrix[2][2])

sum_allelements = 0.0
for i in range(3):
  for j in range(3):
    sum_allelements += confusion_matrix[i][j]

numerator = (confusion_matrix[0][0] + confusion_matrix[1][1] + confusion_matrix[2][2])
final_accuracy = numerator/sum_allelements

print("class1 accuracy ", accuracy1)
print("class2 accuracy ", accuracy2)
print("class3 accuracy", accuracy3)
print("final accuracy  ", final_accuracy)


class1 accuracy  0.7142857142857143
class2 accuracy  0.6842105263157895
class3 accuracy 0.9130434782608695
final accuracy   0.7777777777777778
